# Homework 8: I/O and data structures practice

## Goal

In this assignment we will practice working with files, more practice with dictionaries, and performing some basic geometric computations. Recall earlier in the course I walked through an example of processing retail data where we
had data about a set of products in a store, and the set of baskets that people bought.  We will work
with that and extend it a little for this assignment.  Our working data will be the following:

- _Product inventory_: This will be a table of products where each row corresponds to one product, and each column represents a property of the objects.  Specifically, our product inventory table will have five columns: a unique product ID, a text description of the product, a unit price in dollars, an X coordinate, and a Y coordinate.

- _Sales data_: This will be a table of records from the point of sale system.  Each row will correspond to an item in a basket.  The columns will be the basket ID, the product ID, product quantity, and the product pick up order.  The product pick up order for a basket containing $n$ items will range from 1 to $n$ and corresponds to the order that the customer picked up the products as they went through the store.

We will also be provided additional parameters about the store.  Specifically, we will be given the X,Y
coordinates of the entry door, the checkout stand, and the exit door.  We are going to assume that this
store has only one checkout line and all customers are good at following signs and always use the appropriate door for entry and exit.  We are also going to assume that customers have the magical ability to teleport through shelves so that they can take the shortest path from one product to the next, allowing us to avoid worrying about complex calculations of the distance between any two products.

The coordinates of the entrance, exit, and checkout are:

In [ ]:
store_entrance = (10,0)
store_exit = (90,0)
store_checkout = (50,10)

## Sample I/O code

To get you started, here is code that reads in the inventory data from the CSV file and produces a dictionary of dictionaries.  The outer dictionary maps product IDs as strings to the inner dictionary, and the inner dictionary for each product maps an attribute (e.g., 'unit_price') to its value.  Numerical values are stored as floating point numbers, _not strings_.

In [17]:
import csv

In [18]:
def read_inventory(filename):
    # initialize empty dict
    inventory={}
    
    # open the given file and name the file handle f
    with open(filename) as f:
        # create a CSV reader object from the file
        reader = csv.reader(f)
        
        # advance the reader to skip the first header line
        next(reader)
        
        # for each row in the CSV file, create the appropriate
        # entry in the inventory.  this includes converting the
        # strings for price, x, and y into floats so we can do
        # arithmetic with them later.
        for row in reader:
            inventory[row[0]] = { 'desc':row[1],
                                  'unit_price':float(row[2]),
                                  'x':float(row[3]),
                                  'y':float(row[4]) }
            
        return inventory

In [19]:
inventory = read_inventory('inventory.csv')

Test it: what is the price of product ID 4?

In [31]:
inventory['1']

{'desc': 'banana', 'unit_price': 0.49, 'x': 4.0, 'y': 80.0}

## Part 1: read the basket data

Complete the following function to return a dictionary mapping basket ID to some data structure of your choice to represent the basket contents.

In [35]:
def read_baskets(filename):
    
    baskets={}
    
    with open(filename) as g:
        reader = csv.reader(g)
        next(reader)
        
        for row in reader:
            if row[0] in baskets:
                baskets[row[0]].append({
                                        'product_id':row[1],
                                        'quantity':row[2],
                                        'pickup_order':row[3]})
            else:
                baskets[row[0]] = [{
                                   'product_id':row[1],
                                   'quantity':row[2],
                                   'pickup_order':row[3]}]
                
    return baskets

In [36]:
baskets = read_baskets('baskets.csv')

In [38]:
baskets['1']

[{'pickup_order': '1', 'product_id': '1', 'quantity': '2'},
 {'pickup_order': '2', 'product_id': '3', 'quantity': '1'}]

## Part 2: Calculate the path length of each customer

Given the basket data and inventory data, write a function that calculates the distance traveled by a customer through the store.  Their trip should go entry -> first product -> second product -> ... -> checkout -> exit.  You should assume that the customer takes a straight path from each point to the next.

In [70]:
def customer_trip(inventory, baskets, basket_id):
    import math as math 
    
    basket = baskets[str(basket_id)]
    
    numitems = len(basket)
    prevpos = (10,0)
    dist = 0
    
    for cur in range(1, numitems + 1):
        for item in basket:
            if item ['pickup_order'] == str(cur):
                curpos = inventory[item['product_id']]['x'], inventory[item['product_id']]['y']
                dist = dist + math.sqrt((curpos[0] - prevpos[0])**2 + (curpos[1] - prevpos[1])**2)
                prevpos = curpos
    dist = dist + math.sqrt((50 - prevpos[0])**2 + (10 - prevpos[1])**2)                
    dist = dist + math.sqrt((40)**2 + (-10)**2)
                                            
    return dist

In [71]:
customer_trip(inventory, baskets, 3)

243.55449120845142

## Part 3: Calculate the total price for each basket

Given the basket and inventory data, write a function that calculates the total cost of a basket.

In [48]:
def basket_total(inventory, baskets, basket_id):
    
    basket = baskets[str(basket_id)]
    
    totalprice = 0
    
    for i in basket:
        totalprice += inventory[i['product_id']]['unit_price'] * float(i['quantity'])
        
    
    return totalprice

In [49]:
basket_total(inventory, baskets, 1)

1.97

## Part 4: Calculate the price per unit of distance traveled for all baskets

For each basket we have a distance traveled and a total price.  Write a function that returns a dictionary mapping the basket ID to the price per unit distance travelled.

In [77]:
def price_distance_calculate(inventory, baskets):
    
    rat = {}
    
    for b in baskets:
        rat[b] = basket_total(inventory, baskets, b) / customer_trip(inventory, baskets, b)
        
    return rat 

In [78]:
price_distance_calculate(inventory, baskets)

{'1': 0.0091229257242453,
 '10': 0.032153884764837576,
 '2': 0.06942077638079959,
 '3': 0.09012356902592947,
 '4': 0.07881143685233731,
 '5': 0.07312101528805647,
 '6': 0.01730253662874167,
 '7': 0.055204158287642274,
 '8': 0.00905699668841218,
 '9': 0.06571221581998327}

## Part 5: EXTRA CREDIT.  

### Calculate the difference between the length of the path each customer took versus the shortest path they could have taken.

Each customer may have travelled the store inefficiently.  We would like to know the excess distance each customer covered versus what they could have done had they planned their trip more carefully.  Write a function that calculates the shortest path that a customer could have taken.

In [ ]:
def customer_shortest_trip(inventory, baskets, basket_id):
    # fill me in
    pass